# Real World Use Case: The "Black Image" Bug

**Scenario**: 1 out of your 1 million training images is corrupt (file size 0 bytes). 
**Result**: Method `cv2.imread()` returns `None`. Your training crashes after 3 days.
**Goal**: Build a defensive pipeline.

In [ ]:
import numpy as np

class SafeDataset:
    def __init__(self):
        # Mock a database of 10 images, index 7 is bad
        self.data_ids = range(10)
        
    def load_from_disk(self, idx):
        if idx == 7:
            return None # Corruption
        return np.ones((10, 10))
    
    def __getitem__(self, idx):
        image = self.load_from_disk(idx)
        
        # DEFENSE
        if image is None:
            print(f"Warning: Image {idx} is corrupt. Replacing with backup...")
            # Strategy: Return a different valid image (e.g., index 0)
            # Or simpler: Return a black image (zeros) and ignore label
            return self.__getitem__(0) # Recursive rescue
            
        return image

# Test
ds = SafeDataset()
img = ds[7]
print(f"Recovered Image Shape: {img.shape}")

## Conclusion
In production, never assume data is clean. Always have a fallback in `__getitem__`.